In [1]:
import imgaug.augmenters as iaa
import cv2, os
import glob
import shutil
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from zipfile import ZipFile
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

In [2]:
#JUPYTER VERSION
#comb_path = "../Data/Combined_Images"

#GOOGLE COLAB VERSION
from google.colab import drive
drive.mount('/content/drive')

# Unzipping the dataset file facial-age.zip

combined_images_path = "/content/drive/MyDrive/Data/Combined_Images.zip"

with ZipFile(combined_images_path, 'r') as myzip:
    myzip.extractall("../content/Combined_Images")
    print('Done unzipping Combined_Images.zip')

Mounted at /content/drive
Done unzipping Combined_Images.zip


In [3]:
#1. Load Dataset
def load_dataset(folder_name):
    images = []
    images_path = glob.glob('../content/Combined_Images/{}/*.jpg'.format(str(folder_name)))
    for img_path in images_path:
        img = cv2.imread(img_path)
        img = cv2.resize(img, dsize=(50, 50))
        images.append(img)   
    return images  

In [4]:
def flipping(images):
    #1. --------- Flipping Augmentation ---------
    flipping_augmentation = iaa.Fliplr(1)
    flipped_images = flipping_augmentation(images=images)
    return flipped_images

def rotate_left(images):
    #2. --------- Rotating Left Augmentation ---------
    rotate_left_augmentation = iaa.Affine(translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                                          rotate=(-30, -5),
                                          scale=(0.5, 1.5))
    rotate_left_images = rotate_left_augmentation(images=images)
    return rotate_left_images

def rotate_right(images):
    #3. --------- Rotating Right Augmentation ---------
    rotate_right_augmentation = iaa.Affine(translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                                           rotate=(5, 30),
                                           scale=(0.5, 1.5))
    rotate_right_images = rotate_right_augmentation(images=images)
    return rotate_right_images

def multiply_images(images):
    #4. --------- Multiply --------- to let the image lightner or darker
    multiply_augmentation = iaa.Multiply((0.5, 1.4)) #This is a lightner range
    multiply_images = multiply_augmentation(images=images)
    return multiply_images

def contrast(images):
    #5. --------- Contrast ---------
    contrast_augmentation = iaa.LinearContrast((1.6, 2.4))
    contrast_images = contrast_augmentation(images=images)
    return contrast_images

def blur(images):
    #6. --------- Blur ---------
    blur_augmentation = iaa.GaussianBlur((0.5, 1.5))
    blur_images = blur_augmentation(images=images)
    return blur_images

In [5]:
#Use this to remove folder
import shutil
dirpath = "/content/Combined_Faces_Augmented"
if os.path.exists(dirpath):
    shutil.rmtree(dirpath)

In [6]:
# Making a new folder to save all augmented images as a new dataset.
import os
os.mkdir("Combined_Faces_Augmented")
new_path = "/content/Combined_Faces_Augmented"

In [7]:
#REPEAT THIS PART TO CREATE ALL THE 8 SUBFOLDERS
for num in range(8): #range(1)=range(8)
    os.mkdir('Combined_Faces_Augmented/' + str(num))
    path = '/content/Combined_Faces_Augmented/' + str(num)
    images = load_dataset(str(num))
    tot_imgs = images
    print('---- START IMAGES PROCESSING CLASS {} ----'.format(num))
    for n in range(len(images)):
        if n % 500 == 0:
            print('{} images processed of {}'.format(n, len(images)))
        flp = flipping([images[n]])
        rot_left = rotate_left([images[n]])
        rot_right = rotate_right([images[n]])
        mult = multiply_images([images[n]])
        contr = contrast([images[n]])
        blr = blur([images[n]])
        aug_img = [flp, rot_left, rot_right, mult, contr, blr]
        tot_imgs = tot_imgs + aug_img
    print(len(tot_imgs))    
    print('Image processing for class {} completed\n'.format(num))    
    print('---- START AUGMENTED IMAGES PROCESS CLASS {} ----'.format(num))                 
    for index, img in enumerate(tot_imgs):
        if isinstance(img, list):
            img = img.pop()
        cv2.imwrite(path + '/{}.jpg'.format(index), img)
        if index % 500 == 0:
          print('{} images created of {}'.format(index, len(tot_imgs)))
    print('Augmented images creation process for class {} completed\n'.format(num))        

---- START IMAGES PROCESSING CLASS 0 ----
0 images processed of 4924
500 images processed of 4924
1000 images processed of 4924
1500 images processed of 4924
2000 images processed of 4924
2500 images processed of 4924
3000 images processed of 4924
3500 images processed of 4924
4000 images processed of 4924
4500 images processed of 4924
34468
Image processing for class 0 completed

---- START AUGMENTED IMAGES PROCESS CLASS 0 ----
0 images created of 34468
500 images created of 34468
1000 images created of 34468
1500 images created of 34468
2000 images created of 34468
2500 images created of 34468
3000 images created of 34468
3500 images created of 34468
4000 images created of 34468
4500 images created of 34468
5000 images created of 34468
5500 images created of 34468
6000 images created of 34468
6500 images created of 34468
7000 images created of 34468
7500 images created of 34468
8000 images created of 34468
8500 images created of 34468
9000 images created of 34468
9500 images created 

In [8]:
#Creo la cartella Combined_Images_Augmented.zip e la inserisco sul Google Drive
aug_path = "../content/Combined_Faces_Augmented"
output_filename = '../content/drive/MyDrive/Data/Combined_Images_Augmented'
shutil.make_archive(output_filename, 'zip', aug_path)

'/content/drive/MyDrive/Data/Combined_Images_Augmented.zip'